In [11]:
from AHI_AC import *
res = 0.01

row_AHI = round(120/res)
col_AHI = round(120/res)

AHI_lat = np.linspace(60 - res/2 ,-60 + res/2,row_AHI)
AHI_lon = np.linspace(85 + res/2 ,205 - res/2,col_AHI)

target ='/data01/GEO/SRDATA/'

AHI_VZA,AHI_VAA = read_AHI_VA(res)
Landmask = read_Landmask(res)
AHI_AL = read_AHI_AL(res)
Water_idx = get_water_idx(Landmask)
interpolation = LUT_interpolation(1)
fn1_1, fn2_1, fn3_1 = interpolation.function_dict[interpolation.band]()
interpolation = LUT_interpolation(2)
fn1_2, fn2_2, fn3_2 = interpolation.function_dict[interpolation.band]()
interpolation = LUT_interpolation(3)
fn1_3, fn2_3, fn3_3 = interpolation.function_dict[interpolation.band]()
interpolation = LUT_interpolation(4)
fn1_4, fn2_4, fn3_4 = interpolation.function_dict[interpolation.band]()

In [16]:
def SR_Band1():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','vis','01',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band1)(fn1_1, fn2_1, fn3_1,Landmask,Aerosol_type,OZ,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(12000))
    SR = np.full((12000,12000),np.nan)
    for i in range(12000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(12000,12000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B01_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band1 SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band1 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))

def SR_Band2():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','vis','02',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band2)(fn1_2, fn2_2, fn3_2,Landmask,Aerosol_type,OZ,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(12000))
    SR = np.full((12000,12000),np.nan)
    for i in range(12000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(12000,12000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B02_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band2 SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band2 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))
    
def SR_Band3():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','ext','01',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band3)(fn1_3, fn2_3, fn3_3,Landmask,Aerosol_type,WV,OZ,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(12000))
    SR = np.full((12000,12000),np.nan)
    for i in range(12000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(12000,12000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B03_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band3 SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band3 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))
def SR_Band4():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','vis','03',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band4)(fn1_4, fn2_4, fn3_4,Landmask,Aerosol_type,WV,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(12000))
    SR = np.full((12000,12000),np.nan)
    for i in range(12000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(12000,12000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B04_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band4 SR finished')
    remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band4 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))

In [ ]:

date_start = '2018-09-01 00:00'.format(d)
date_end = '2018-09-30 09:00'.format(d)
date_t = dt.timedelta(minutes=10)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
print("Start {} ".format(date_start[:-6]))
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
CAMS_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS()
CAMS_AERO_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS_AERO()
while date_time_now <= date_e:

    date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
    hour = int(date[8:10])

    if hour <= 9:
        print("start processing {}".format(date))
    # make dir
        folder_original = target + date +'_original/'
        folder_AC = target+'{}/{}/{}/'.format(YYYY,MM,DD)
        mkdir(folder_original)
        mkdir(folder_AC)

        S_Read_AHI_time = T.time()

        # Solar angle
        print('Start read solar angle')
        S_ANGLE_time = T.time()
        AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
        RAA = abs(AHI_SAA - AHI_VAA)
        RAA[RAA>180]=360-RAA[RAA>180]
        print('Finish read solar angle')
        E_ANGLE_time = T.time()

        # Atmosphereic data
        print('Start read CAMS')
        S_ATMOS_time = T.time()
        OZ,WV,AOT550 = CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).CAMS_Temporal(CAMS_ds)
        Aerosol_type = CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).CAMS_AERO_Temporal(CAMS_AERO_ds)
        print('Finish read CAMS')

        E_ATMOS_time = T.time()
        print('Read Atmospheric data cost : {}'.format(E_ATMOS_time-S_ATMOS_time))

        SR_Band1()
        SR_Band2()
        SR_Band3()
        SR_Band4()


        date_time_now = date_time_now + date_t